In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms 
import sklearn
import sklearn.metrics as metrics
import numpy as np

import pandas as pd
from torch.utils.data import Dataset
import os 
from PIL import Image

In [2]:
class CUB (Dataset):
    '''
    parts 
        part_locs 
        parts 
    images 
        200 classes
    classes.txt
    train_test_split.txt
    image_class_labels.txt
    
    '''
    def __init__(self, root_dir, transform = None):
        self.root_dir = root_dir # D:\TorchProject\dataset\cub\CUB_200_2011
        self.transform = transform
        self.image_dir = os.path.join(root_dir,'images')
        self.image_mapping = self._load_image_mapping()
        self.label_mapping = self._load_label_mapping()
        self.bbox_mapping = self._load_bbox_mapping ()
        self.attribute_mapping = self._load_attribute_mapping()
        self.part_bbox_mapping = self._load_part_bbox_mapping()
        self.part_locs_mapping = self._load_part_locs_mapping()

    def _load_part_locs_mapping (self):
        part_locs_mapping = {}
        part_locs = os.path.join(self.root_dir, 'parts', 'part_locs.txt')
        with open(part_locs, 'r') as file:
            for line in file:
                image_id, part_id, x, y, visible = line.strip().split()
                if visible == '1':
                    if image_id not in part_locs_mapping:
                        part_locs_mapping[image_id] = {}
                    part_locs_mapping[image_id][part_id] = [float(x), float(y)]
                    part_locs_mapping[image_id]['visible'] = float(visible)
        return part_locs_mapping

    def _load_label_mapping (self): # why "_load_..." ? --> chi dung trong class nay, k call dc tu ben ngoai
        label_mapping = {}
        label_file = os.path.join(self.root_dir,'image_class_labels.txt')
        with open(label_file, 'r') as file:
            for line in file:
                image_id, label_id = line.strip().split()
                label_mapping[image_id] = int(label_id)
        return label_mapping

    def _load_image_mapping (self):
        image_mapping = {}
        image_file = os.path.join(self.root_dir,'images.txt')
        with open(image_file, 'r') as file:
            for line in file:
                image_id, image_name = line.strip().split()
                image_mapping[image_id] = image_name
        return image_mapping

    def _load_bbox_mapping (self):
        bbox_mapping = {}
        bbox_file = os.path.join(self.root_dir,'bounding_boxes.txt')
        with open(bbox_file, 'r') as file:
            for line in file:
                image_id, x, y, width, height = line.strip().split()

                bbox_mapping[image_id] = [float (x), float (y), float (width), float (height)]
        return bbox_mapping

    def _load_attribute_mapping(self):
        attribute_mapping = {}
        attribute_file = os.path.join(self.root_dir, 'attributes', 'image_attribute_labels.txt')
        with open(attribute_file, 'r') as file:
            for line in file:
                if len((line.strip().split())) != 5:
                    image_id, attribute_id, is_present, _, _, _ = line.strip().split()
                else:
                    image_id, attribute_id, is_present, _, _= line.strip().split()
                if image_id not in attribute_mapping:
                    attribute_mapping[image_id] = []
                attribute_mapping[image_id].append(attribute_id)
        return attribute_mapping

    def _load_part_bbox_mapping(self):
        part_bbox_mapping = {}
        part_bbox_file = os.path.join(self.root_dir, 'parts', 'part_locs_test.txt')
        with open(part_bbox_file, 'r') as file:
            for line in file:
                image_id, part_id, x, y, visible = line.strip().split()
                if visible == '1':
                    if image_id not in part_bbox_mapping:
                        part_bbox_mapping[image_id] = {}
                    part_bbox_mapping[image_id][part_id] = [float(x), float(y)]
        return part_bbox_mapping

    def __len__(self):
        return len(self.image_mapping)

    def __getitem__(self, index):
#        label = self.label_mapping[str(index+1)]
#        bbox = self.bbox_mapping[str(index+1)]
#        attribute = self.attribute_mapping.get(str(index), [])
#        part_bbox = self.part_bbox_mapping.get(str(index), {})
#        part_locs = self.part_bbox_mapping.get(str(index), [])
#        image_name = self.image_mapping[str(index)]
        
        label = self.label_mapping[str(index+1)]
        part_locs = self.part_bbox_mapping.get(str(index), [])
        image_name = self.image_mapping[str(index)]
        print(image_name)

        image= Image.open(os.path.join(self.image_dir,image_name)).convert('RGB')

        if self.transform:
           image = self.transform(image)
        return (image, label, part_locs)
#        return (image, label, bbox, attribute, part_bbox, part_locs)

In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch.utils.data import DataLoader
import os

def visualize_bounding_box(image, bbox, part_bbox, part_locs):
        # Create figure and axes
        fig, ax = plt.subplots(1)

        # Display the image
        ax.imshow(image)

    # Extract bounding box coordinates
        x, y, width, height = bbox
        #x, y, width, height = box

        # Create a rectangle patch
        rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='r', facecolor='none')

        # Add the rectangle to the plot
        ax.add_patch(rect)

        #plt.show()


        # Iterate over part bounding boxes and draw rectangles
        for part_id, part_bb in part_bbox.items():
            x_pb, y_pb = part_bb
            width_pb, height_pb = 5, 5  # Modify the width and height as per your requirement

            rect_pb = patches.Rectangle((x_pb, y_pb), width_pb, height_pb, linewidth=1, edgecolor='b', facecolor='none')

            ax.add_patch(rect_pb)

            # Show the plot
        plt.show()


# Example usage

root_dir = 'D:\TorchProject\dataset\cub\CUB_200_2011'
# run dataser == __init__
dataset = CUB(root_dir) # --> load sample to check
dataloader = DataLoader(dataset, batch_size=2, shuffle=False) #divide dataset by batch

# call len  == __len__
print (len(dataset))

# indexing in  dataset == __getitem__
# Get the image and bounding box at the specified index
#image,label,bbox,attribute,part_bbox = dataset[10]

for i in range (1,10):
#    image,label,bbox,attribute,part_bbox, part_locs = dataset[i]
    image,label,bbox, part_bbox,part_locs = dataset[i]

    print (label,bbox,part_locs)

    # Visualize all bounding boxes
    visualize_bounding_box(image, bbox, part_bbox, part_locs)

11788
001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg


ValueError: not enough values to unpack (expected 5, got 3)

In [9]:
class simple_graph (nn.Module):
    def __init__ (self):
        super (simple_graph, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3) 
        self.d1 = nn.Linear(224*224*32, 128)
        self.d2 = nn.Linear(128, 10)
        #x = nn.Linear(5,5)

    def forward (self, x):
        x = self.conv1(x)
        x = F.relu(x)

        x = x.flatten(start_dim=1)

        x = self.d1(x)
        x = F.relu(x)

        logits = self.d2(x)
        out = F.softmax(logits, dim = 1)

        return out

